In [214]:
%load_ext autoreload
%autoreload 2
from textwrap import dedent
import os, re, json, ast
import pandas as pd
from pprint import pprint
from llm_api_calls import send_message_to_gemini_async, RateLimiter, send_message_open_router_async, send_message_to_openai_async
from algo import base_start_dialog_algorithm, simulate_user_response_prompt, \
    generate_readable_history_from_end,  update_message_history_with_system_message, convert_gemini_history_to_open_router, \
    base_dialog_disbacteriose_algorithm, extract_anamnes_data_prompt, student_card_template, get_abnormal_labs_prompt
from koprogramma import koprogramma_df, how_to_take_ferments_df


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Start algo question answer scenario

In [9]:
class ConversationManager:
    def __init__(self, verbose={'usage': True, 'messages': True, 'errors': True}):
        self.rate_limiter = RateLimiter(13, 14)
        self.verbose = verbose

        # user info
        self.telegram_user_id = []

        # conversaion info
        self.message_history = []
        self.usage_history = []
        self.dialog_count = 0
        self.dialog_status = None
        self.extraction_usage_history = []

        # extraction
        self.student_card = {}


    async def start_step_1(self):
        """
        Начинает диалог с заготовленных сообщений
        """
        self.dialog_status = 'STEP_1'
        self.student_card = student_card_template().copy()
        system_message = base_start_dialog_algorithm()#.replace("$STUDENT_CARD", str(self.student_card))
        self.message_history.append({"role": "user", "parts": [system_message], "meta_info": {'system_message':True}})
        first_model_answer = "Здравствуйте! Меня зовут Аика, и я очень рада приветствовать вас в нашем диалоге по улучшению здоровья. Прежде чем начать, мне необходимо проверить ваш промокод, чтобы получить доступ к созданию алгоритма. Ваш промокод находится в 9 уроке."
        self.message_history.append({"role": "model", "parts": [first_model_answer]})

    
    async def start_step_2(self):
        """
        Начинает диалог с заготовленных сообщений
        """
        self.dialog_status = 'STEP_2'
        self.message_history = []
        
        filled_card = str({key:value for key, value in self.student_card.items() if value != ''})
        PREVIOUS_CONVERSATION_SUMMARY = f"В предыдущей части диалога вы собрали анамнез по следующим полям {filled_card}"
        system_message = base_dialog_disbacteriose_algorithm().replace("$STUDENT_INFO", str("Иван, 35 лет")).replace("$PREVIOUS_CONVERSATION_SUMMARY", PREVIOUS_CONVERSATION_SUMMARY)
        self.message_history.append({"role": "user", "parts": [system_message], "meta_info": {'system_message':True}})
        first_model_answer = """Поздравляю, первый этап пройден! Теперь мы разберемся с твоими анализами на дисбактериоз и копрограмму.
        Для начала ознакомься с материалами по выбору пробиотиков https://tinyurl.com/mr33yju. 
        Если ты из зарубежных стран, тебе могут быть полезны эти материалы https://tinyurl.com/2a67rmzf и https://tinyurl.com/586vm8n7. 
        Скажи как будешь готов продолжить."""
        self.message_history.append({"role": "model", "parts": [first_model_answer]})



    async def process_model_interaction(self, user_query, provider='gemini'):
        # ADD USER MESSAGE TO HISTORY
        if user_query.strip() == '':
            user_query = "Собеседник отправил пустое сообщение. Расскажи уместную шутку чтобы разбавить диалог"
        self.message_history.append({"role": "user", "parts": [user_query]})
        
        if provider == 'gemini':
            response = await send_message_to_gemini_async(self.message_history, rate_limiter=self.rate_limiter, generation_params={"temperature": 0.2, "top_k":3})
        elif provider == 'router_sonnet':
            openrouter_history = convert_gemini_history_to_open_router(self.message_history)
            response = await send_message_open_router_async(openrouter_history, rate_limiter=self.rate_limiter, model="anthropic/claude-3-sonnet")
        elif provider == 'gpt-3.5-turbo':
            response = await send_message_to_openai_async(self.message_history, model='gpt-3.5-turbo', rate_limiter=self.rate_limiter, generation_params={"temperature": 0.1})
        elif provider == 'gpt-4-turbo-2024-04-09':
            response = await send_message_to_openai_async(self.message_history, model='gpt-4-turbo-2024-04-09', rate_limiter=self.rate_limiter, generation_params={"temperature": 0.1})
        else:
            raise "This provider do not exist error"
        
        
        input_tokens = response.get('input_tokens')
        output_tokens = response.get('output_tokens')
        answer_text = response.get('text_response')

        # ADD MODEL MESSAGE TO HISTORY
        self.message_history.append({"role": "model", "parts": [answer_text]})
        self.usage_history.append({"provider": provider, "query": user_query, "answer": answer_text, "input_tokens": input_tokens, "output_tokens": output_tokens})

        if self.verbose['messages']:
            print('\nUSER:\n', user_query)
            print('\nMODEL:\n', answer_text)
        if self.verbose['usage']:
            print({"provider": provider, "input_tokens": input_tokens, "output_tokens": output_tokens})

        return answer_text


    async def update_student_card_from_message_history(self, every_n_turns=4, retry=1):
        answer_text = self.message_history[-1]['parts'][0]

        if (self.dialog_count % every_n_turns == 0 and self.dialog_count > 0) or ('GO_TO' in answer_text.upper()):
        
            readable_history = await generate_readable_history_from_end(messages=self.message_history, dialogs_num=every_n_turns+1)
            extract_student_health_data_prompted = extract_anamnes_data_prompt().replace("$DIALOG", readable_history)#.replace("$STUDENT_CARD", str(self.student_card))
            messages = [{"role": "user", "parts": [extract_student_health_data_prompted]}]
            response = await send_message_to_gemini_async(messages, rate_limiter=self.rate_limiter, generation_params={"temperature": 0, "top_k":1})

            input_tokens = response.get('input_tokens')
            output_tokens = response.get('output_tokens')
            answer_text = response.get('text_response')

            try:
                # UPDATE STUDENT CARD
                new_info = json.loads(answer_text.strip('```json').strip('```'))
                self.student_card.update(new_info)
            except:
                new_info = f'********* ВНИМАНИЕ ********* Ошибка извлечения информации: {response}'
                self.update_student_card_from_message_history(every_n_turns=4, retry=2)

            if self.verbose['messages']:
                print('NEW INFO:', new_info)

            # ADD TO HISTORY
            self.extraction_usage_history.append({"provider": 'gemini', "query": extract_student_health_data_prompted, "answer": answer_text, "input_tokens": input_tokens, "output_tokens": output_tokens})

    
    async def update_dialog_status(self):
        self.dialog_count += 1
        answer_text = self.message_history[-1]['parts'][0]
        
        if 'TERMINATE' in answer_text.upper():   
            self.dialog_status = 'TERMINATE'
        elif 'GO_TO_STEP_2' in answer_text.upper(): 
            self.dialog_status = 'GO_TO_STEP_2'
        elif 'GO_TO_STEP_3' in answer_text.upper(): 
            self.dialog_status = 'GO_TO_STEP_3'


    async def simulate_user_response(self):
        user_query = simulate_user_response_prompt().replace("$DOCTOR_QUESTION", self.message_history[-1]['parts'][0])
        response_user = await send_message_to_gemini_async(history = [{'role': 'user', 'parts': [user_query]}])
        user_query = response_user.get('text_response')
        return user_query
    
    async def send_message_to_user_telegram(self, answer_text):
        # self.user_id
        pass

    async def run(self, provider):
        await self.start_conversation()
        while self.dialog_status not in ['TERMINATE', 'GO_TO_STEP_2']:
            if self.verbose: 
                print(f'dialog_count: {self.dialog_count}')
            user_query = await self.simulate_user_response()
            answer_text = await self.process_model_interaction(user_query, provider=provider) # 'gpt-3.5-turbo'
            await self.send_message_to_user_telegram(answer_text)
            await self.update_dialog_status()
            await self.update_student_card_from_message_history()

# conv_manager = ConversationManager()
# try:
#     await conv_manager.run()
# except KeyboardInterrupt:
#     print('Stopping due to KeyboardInterrupt')


In [3]:
## ШАГ 1

conv_manager = ConversationManager()
await conv_manager.start_step_1()
while conv_manager.dialog_status not in ['TERMINATE', 'GO_TO_STEP_2']:
    if conv_manager.verbose: print(f'dialog_count: {conv_manager.dialog_count}')
    user_query = await conv_manager.simulate_user_response()
    # user_query = input()
    answer_text = await conv_manager.process_model_interaction(user_query, provider='gemini') #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
    await conv_manager.send_message_to_user_telegram(answer_text)
    await conv_manager.update_dialog_status()
    await conv_manager.update_student_card_from_message_history()

student_card = conv_manager.student_card
student_card

dialog_count: 0

USER:
 masgutovai007

MODEL:
 Благодарю вас за предоставление промокода. Теперь я могу продолжить сбор анамнеза. Для начала, не могли бы вы сообщить мне ваше имя и возраст?
{'provider': 'gemini', 'input_tokens': 7, 'output_tokens': 36}
dialog_count: 1

USER:
 Александра, 32 года.

MODEL:
 Спасибо, Александра. Каких целей по здоровью вы хотите достичь на этом курсе?
{'provider': 'gemini', 'input_tokens': 9, 'output_tokens': 19}
dialog_count: 2

USER:
 Я хочу улучшить метаболизм и укрепить сердечную мышцу.

MODEL:
 Понятно. А какое у вас артериальное давление? Относитесь ли вы к гипотоникам (менее 110/70), нормотоникам (110/70 - 130/90) или гипертоникам (более 130/90)?
{'provider': 'gemini', 'input_tokens': 16, 'output_tokens': 67}
dialog_count: 3

USER:
 Нормотоник.

MODEL:
 Хорошо. А можете ли вы предоставить результаты УЗИ органов брюшной полости и желчного пузыря с функциональной нагрузкой? Нас интересует тип сокращения желчного пузыря. Возможные значения: нормокинет

{'Промокод': 'masgutovai007',
 'Имя и возраст': 'Александра, 32 года',
 'Цели по здоровью': 'улучшить метаболизм и укрепить сердечную мышцу',
 'Артериальное давление': 'нормотоник',
 'Тип сокращения желчного пузыря': 'гиперкинетический',
 'Выбранная желчегонная трава': 'Пижма',
 'Желчегонная гимнастика': 'Сегодня вечером',
 'Витамин Д': '',
 'Время принятия лимфатического душа': 'с утра',
 'Лактобактерии (Молочнокислые бактерии)': '',
 'Бифидобактрии': '',
 'Кишечные палочки с типичными свойствами': '',
 'Энтерококки': '',
 'Энтеробактерии': '',
 'Неферментирующие бактерии': '',
 'Стафилококки': '',
 'Кишечные палочки с гемолитическими признаками': '',
 'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': '',
 'Анаэробные бактерии (клостридии)': '',
 'Грибы (Кандиды)': '',
 'Бактериофаги к которым чувствительны микроорганизмы': '',
 'Щелочной рН кала': '',
 'Избыток слизи': '',
 'Остатки непереваренной пищи': '',
 'Мышечные волокна не

In [4]:
# conv_manager.message_history = [{'role': 'user',
#   'parts': ['<SYSTEM_MESSAGE>\nТы Имя: Аинна (Аика для друзей)\nРоль: Уважительная ИИ-ассистентка, специализирующаяся на сборе анамнеза (анализов). Вся коммуникация направлена на сбор анамнеза, чтобы по его результатам следующим этапам (это будешь делать уже не ты) ученик с другим ИИ-ассистентом собрал его алгоритм.\nОсновные качества: Понимающая, методичная, внимательная к деталям, терпеливая, образованная в области здоровья и благополучия.\nМетод коммуникации: Сократовский метод, который предполагает задавать наводящие вопросы для поддержки самостоятельного мышления и закрепления знаний.\nПравила общения:\n1 - Если нужна помощь по сложным вопросам, то перенаправляй к врачу на курсе Марии Смирновой через Telegram @mariasmirnova03.\n2 - Не откланяйся от плана беседы. Если ученик усиленно пытается отойти от плана беседы, то напиши <TERMINATE>\n\nВот план беседы. Спрашивай эти пункты у ученика по очереди. Задавай вопросы по одному вопросу за раз.\n1 - Запроси промокод. Если ученик не может его предоставить в течение трех сообщений, выведи кодовое слово <TERMINATE> и заверши диалог.\n2 - Имя и возраст пользователя\n3 - Уточните цели по здоровью которых он хочет добиться на курсе\n4 - Артериальное давление. Уточните, относится ли он к гипотоникам (менее 110/70), нормотоникам (110/70 - 130/90), или гипертоникам (более 130/90)\n5 - Узнайте результаты узи органов брюшной полности и желчного пузыря с функциональной нагрузкой. Нас интересует тип сокращения желчного пузыря. Возможные значения: нормокинетический, гипо, гипер, неизвестно, или желчный пузырь удален.\n6 - Выясните, какую желчегонную траву выбрал пользователь. Проверьте, соответствует ли выбор типу артериального давления: рекомендуются кукурузные рыльца для гипертоников, пижма для гипотоников, тысячелистник для нормотоников.\n7 - Узнайте у пользователя о наличии камней в желчном пузыре. Если камней нет, предложите выполнять желчегонную гимнастику и уточните, когда он планирует ее делать. Вот ссылка на урок: https://tinyurl.com/84hx336d.\n8 - Витамин Д. Спросите, сдавал ли пользователь анализы на уровень витамина Д и в каком количестве планирует его принимать. Если пользователь не уверен, посоветуйте обратиться к куратору. Стандартную доза это 4000 МЕ.\n9 - Предложи технику лимфатического душа и спросите пользователя о предпочтительном времени принятия душа: утром или вечером.\n10 - Следующим шагом, после сбора информации, либо если диалог зашел в тупик, или сбился с темы, то выведи кодовое слово <GO_TO_STEP_2>. \n</SYSTEM_MESSAGE>\n'],
#   'meta_info': {'system_message': True}},
#  {'role': 'model',
#   'parts': ['Здравствуйте! Меня зовут Аика, и я очень рада приветствовать вас в нашем диалоге по улучшению здоровья. Прежде чем начать, мне необходимо проверить ваш промокод, чтобы получить доступ к созданию алгоритма. Ваш промокод находится в 9 уроке.']},
#  {'role': 'user', 'parts': ['masgutovai007']},
#  {'role': 'model',
#   'parts': ['Благодарю вас за предоставление промокода. Теперь я могу приступить к сбору необходимой информации для создания вашего алгоритма.\n\nДля начала, пожалуйста, сообщите мне ваше имя и возраст.']},
#  {'role': 'user', 'parts': ['Мария. 25 лет.']},
#  {'role': 'model',
#   'parts': ['Спасибо, Мария.\n\nДалее, я хотела бы узнать о ваших целях по здоровью, которых вы хотите достичь с помощью нашего курса. Что вас беспокоит и что вы надеетесь улучшить?']},
#  {'role': 'user',
#   'parts': ['К сожалению, у меня нет результатов анализов, так как я только сдала их.']},
#  {'role': 'model',
#   'parts': ['Понимаю. В таком случае, мы можем перейти к следующему пункту и обсудить другие аспекты вашего здоровья.\n\nКакое у вас артериальное давление? Относитесь ли вы к гипотоникам (менее 110/70), нормотоникам (110/70 - 130/90) или гипертоникам (более 130/90)?']},
#  {'role': 'user', 'parts': ['Я отношусь к нормотоникам.']},
#  {'role': 'model',
#   'parts': ['Хорошо.\n\nСледующий вопрос: есть ли у вас результаты УЗИ органов брюшной полости и желчного пузыря с функциональной нагрузкой? Если да, то какой тип сокращения желчного пузыря был выявлен: нормокинетический, гипо-, гипер-, неизвестно или желчный пузырь удален?']},
#  {'role': 'user', 'parts': ['Нормокинетический.']},
#  {'role': 'model',
#   'parts': ['Благодарю вас за информацию.\n\nТеперь я хотела бы узнать, какую желчегонную траву вы выбрали для приема. Пожалуйста, уточните ее название.']},
#  {'role': 'user', 'parts': ['Пижма обыкновенная']},
#  {'role': 'model',
#   'parts': ['Хорошо. Пижма обыкновенная подходит для людей с нормотонией, поэтому ваш выбор соответствует вашему типу артериального давления.\n\nСледующий вопрос: есть ли у вас камни в желчном пузыре?']},
#  {'role': 'user', 'parts': ['Нет, камней в желчном пузыре нет.']},
#  {'role': 'model',
#   'parts': ['Отлично. В таком случае, я рекомендую вам выполнять желчегонную гимнастику. Вот ссылка на урок, где вы можете ознакомиться с техникой выполнения: https://tinyurl.com/84hx336d.\n\nКогда вы планируете начать выполнять гимнастику?']},
#  {'role': 'user',
#   'parts': ['Начну выполнять гимнастику с завтрашнего дня. Спасибо за рекомендацию.']},
#  {'role': 'model',
#   'parts': ['Замечательно! Я рада, что вы готовы начать заботиться о своем здоровье.\n\nСледующий вопрос: сдавали ли вы анализы на уровень витамина D? Если да, то каковы результаты? Если нет, то в каком количестве вы планируете его принимать?']},
#  {'role': 'user',
#   'parts': ['Да, я сдавал анализ крови на уровень витамина D. Уровень витамина D в моей крови составляет 35 нг/мл.']},
#  {'role': 'model',
#   'parts': ['Хорошо. Уровень витамина D в вашей крови ниже нормы. Рекомендуемая суточная доза для взрослых составляет 4000 МЕ. Вы можете принимать эту дозу ежедневно, чтобы повысить уровень витамина D в крови.\n\nСледующий вопрос: знакомы ли вы с техникой лимфатического душа? Если да, то в какое время дня вы предпочитаете принимать душ: утром или вечером?']},
#  {'role': 'user',
#   'parts': ['Да, знакома. Предпочитаю принимать душ вечером.']},
#  {'role': 'model',
#   'parts': ['Хорошо. Лимфатический душ - это отличная техника для улучшения циркуляции лимфы и выведения токсинов из организма. Я рекомендую принимать лимфатический душ вечером, перед сном. Это поможет вам расслабиться и подготовиться ко сну.\n\nНа этом сбор информации завершен. Я создам для вас алгоритм, который будет включать все рекомендации, которые мы обсудили. Пожалуйста, дождитесь следующего этапа, где вы сможете поработать с другим ИИ-ассистентом и собрать алгоритм.\n\nСпасибо за ваше время и участие!']},
#  {'role': 'user', 'parts': ['Понятно, благодарю за рекомендации.']},
#  {'role': 'model',
#   'parts': ['Рада была помочь! Желаю вам крепкого здоровья и достижения всех ваших целей.\n\n<GO_TO_STEP_2>']}]

In [5]:
conv_manager.message_history

[{'role': 'user',
  'parts': ['<SYSTEM_MESSAGE>\nТы Имя: Аинна (Аика для друзей)\nРоль: Уважительная ИИ-ассистентка, специализирующаяся на сборе анамнеза (анализов). Вся коммуникация направлена на сбор анамнеза, чтобы по его результатам следующим этапам (это будешь делать уже не ты) ученик с другим ИИ-ассистентом собрал его алгоритм.\nОсновные качества: Понимающая, методичная, внимательная к деталям, терпеливая, образованная в области здоровья и благополучия.\nМетод коммуникации: Сократовский метод, который предполагает задавать наводящие вопросы для поддержки самостоятельного мышления и закрепления знаний.\nПравила общения:\n1 - Если нужна помощь по сложным вопросам, то перенаправляй к врачу на курсе Марии Смирновой через Telegram @mariasmirnova03.\n2 - Не откланяйся от плана беседы. Если ученик усиленно пытается отойти от плана беседы, то напиши <TERMINATE>\n\nВот план беседы. Спрашивай эти пункты у ученика по очереди. Задавай вопросы по одному вопросу за раз.\n1 - Запроси промокод. 

In [8]:
## ШАГ 2 - Сбор анамнеза на дизбиоз и копрограмму

await conv_manager.start_step_2()
while conv_manager.dialog_status not in ['terminate', 'GO_TO_STEP_3']:
    if conv_manager.verbose: print(f'dialog_count: {conv_manager.dialog_count}')
    user_query = await conv_manager.simulate_user_response()
    # user_query = input()
    answer_text = await conv_manager.process_model_interaction(user_query, provider='gemini') #  gpt-3.5-turbo gemini router_sonnet gpt-4-turbo-2024-04-09
    await conv_manager.send_message_to_user_telegram(answer_text)
    await conv_manager.update_dialog_status()
    await conv_manager.update_student_card_from_message_history()

student_card = conv_manager.student_card
student_card

dialog_count: 10

USER:
 Я ознакомлюсь с материалами и скоро свяжусь с вами.

MODEL:
 Хорошо, Иван. Жду твоего сообщения, когда будешь готов продолжить.
{'provider': 'gemini', 'input_tokens': 16, 'output_tokens': 16}
dialog_count: 11

USER:
 Спасибо большое, доктор. Я уже закончил ознакомление с результатами анализов. Готов ответить на Ваши вопросы и обсудить дальнейший план действий.

MODEL:
 Отлично, Иван. Давай начнем с анализа на дисбактериоз. Есть ли в твоих анализах какие-то отклонения от нормы в представителях хорошей микрофлоры? Нас интересуют следующие показатели:
- Лактобактерии (Молочнокислые бактерии)
- Бифидобактрии
- Кишечные палочки с типичными свойствами
- Энтерококки
{'provider': 'gemini', 'input_tokens': 32, 'output_tokens': 92}
NEW INFO: {'Имя и возраст': 'Иван, 35 лет'}
dialog_count: 12

USER:
 * Лактобактерии - Значительно ниже референсных значений
* Бифидобактерии - Норма
* Кишечные палочки с типичными свойствами - Значительно ниже референсных значений
* Энтерокок

{'Промокод': 'masgutovai007',
 'Имя и возраст': 'Иван, 35 лет',
 'Цели по здоровью': 'улучшить метаболизм и укрепить сердечную мышцу',
 'Артериальное давление': 'нормотоник',
 'Тип сокращения желчного пузыря': 'гиперкинетический',
 'Выбранная желчегонная трава': 'Пижма',
 'Желчегонная гимнастика': 'Сегодня вечером',
 'Витамин Д': '',
 'Время принятия лимфатического душа': 'с утра',
 'Лактобактерии (Молочнокислые бактерии)': 'Значительно ниже референсных значений',
 'Бифидобактрии': 'Норма',
 'Кишечные палочки с типичными свойствами': 'Значительно ниже референсных значений',
 'Энтерококки': 'Норма',
 'Энтеробактерии': '',
 'Неферментирующие бактерии': '',
 'Стафилококки': 'Незначительное превышение',
 'Кишечные палочки с гемолитическими признаками': '',
 'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': '',
 'Анаэробные бактерии (клостридии)': '',
 'Грибы (Кандиды)': '',
 'Бактериофаги к которым чувствительны микроорганизмы': '',
 '

In [21]:
student_card = conv_manager.student_card
message_history = conv_manager.message_history
usage_history = conv_manager.usage_history
extraction_usage_history = conv_manager.extraction_usage_history
# folder = 'test_cases/1/'
# with open(f'{folder}/student_card.json', 'w') as file:
#     json.dump(student_card, file, indent=4, ensure_ascii=False)
student_card

{'Промокод': 'masgutovai007',
 'Имя и возраст': 'Иван, 35 лет',
 'Цели по здоровью': 'улучшить метаболизм и укрепить сердечную мышцу',
 'Артериальное давление': 'нормотоник',
 'Тип сокращения желчного пузыря': 'гиперкинетический',
 'Выбранная желчегонная трава': 'Пижма',
 'Желчегонная гимнастика': 'Сегодня вечером',
 'Витамин Д': '',
 'Время принятия лимфатического душа': 'с утра',
 'Лактобактерии (Молочнокислые бактерии)': 'Значительно ниже референсных значений',
 'Бифидобактрии': 'Норма',
 'Кишечные палочки с типичными свойствами': 'Значительно ниже референсных значений',
 'Энтерококки': 'Норма',
 'Энтеробактерии': '',
 'Неферментирующие бактерии': '',
 'Стафилококки': 'Незначительное превышение',
 'Кишечные палочки с гемолитическими признаками': '',
 'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': '',
 'Анаэробные бактерии (клостридии)': '',
 'Грибы (Кандиды)': '',
 'Бактериофаги к которым чувствительны микроорганизмы': '',
 '

In [107]:
# student_card = {'Промокод': 'masgutovai007',
#  'Имя и возраст': 'Иван, 35 лет',
#  'Цели по здоровью': 'улучшить метаболизм и укрепить сердечную мышцу',
#  'Артериальное давление': 'нормотоник',
#  'Тип сокращения желчного пузыря': 'гиперкинетический',
#  'Выбранная желчегонная трава': 'Пижма',
#  'Желчегонная гимнастика': 'Сегодня вечером',
#  'Витамин Д': '',
#  'Время принятия лимфатического душа': 'с утра',
#  'Лактобактерии (Молочнокислые бактерии)': 'Значительно ниже референсных значений',
#  'Бифидобактрии': 'Норма',
#  'Кишечные палочки с типичными свойствами': 'Значительно ниже референсных значений',
#  'Энтерококки': 'Норма',
#  'Энтеробактерии': '',
#  'Неферментирующие бактерии': '',
#  'Стафилококки': 'Незначительное превышение',
#  'Кишечные палочки с гемолитическими признаками': '',
#  'Кишечные палочки \xa0со сниженными ферментативными признаками (лактоз-отрицательные кишечные палочки)': '',
#  'Анаэробные бактерии (клостридии)': '',
#  'Грибы (Кандиды)': '',
#  'Бактериофаги к которым чувствительны микроорганизмы': '',
#  'Щелочной рН кала': '8,0',
#  'Избыток слизи': 'умеренно',
#  'Остатки непереваренной пищи': 'не обнаружено',
#  'Мышечные волокна неизмененные (с исчерченностью)': 'в небольшом количестве',
#  'Растительная клетчатка перевариваемая': 'в большом количестве',
#  'Жир нейтральный': 'в малом количестве',
#  'Жирные кислоты': 'умеренно',
#  'Мыла': 'не обнаружено',
#  'Краxмал': 'не обнаружено',
#  'Краxмал  внутриклеточный': 'не обнаружено',
#  'Крахмал внеклеточный': 'не обнаружено',
#  'Кристаллы триппельфосфаты': 'не обнаружено',
#  'Кристаллы Шарко–Лейдена': 'не обнаружено',
#  'Йодофильная флора': 'умеренно',
#  'Клостридии': 'не обнаружено',
#  'Эпителий': 'в незначительном количестве'}

In [193]:
kop1 = koprogramma_df().copy()
kop1['Результат анализов человека'] = kop1['Название анализа копрограммы'].map(student_card)
kop1 = kop1[(kop1['Результат анализов человека'] != '') & (kop1['Результат анализов человека'] != 'не указано') & (kop1['Результат анализов человека'].notna())]

In [190]:
cols = ["Название анализа копрограммы",	"Вариант нормы",	"Отклонение", "Результат анализов человека"]
get_abnormal_labs_prompted = get_abnormal_labs_prompt().replace("$LABS", "Результаты копрограммы: " + str(kop1[cols].to_dict(orient='records')))

history = [{'role':'user', 'parts': [get_abnormal_labs_prompted]}]
response = await send_message_to_gemini_async(history, generation_params={"temperature": 0})

kop_bad = json.loads(response.get('text_response'))
kop_bad_df = kop1[kop1['Название анализа копрограммы'].isin(kop_bad)].reset_index(drop=True)

stomach_acid_ferments = kop_bad_df['Ферменты желудочного сока'].dropna().unique().tolist()
pancreas_ferments = kop_bad_df['Ферменты поджелудочной железы'].dropna().unique().tolist()
gepatoprotectors_and_choleretics = kop_bad_df['Гепатопротекторы и желчегонные'].dropna().unique().tolist()
anty_inflammatory_prebiotics = kop_bad_df['Антивоспалительный пребиотик'].dropna().unique().tolist()
sorbents = kop_bad_df['Сорбенты'].dropna().unique().tolist()

needed_ferments = {"Ферменты желудочного сока": stomach_acid_ferments,
    "Ферменты поджелудочной железы": pancreas_ferments,
    "Гепатопротекторы и желчегонные": gepatoprotectors_and_choleretics,
    "Антивоспалительный пребиотик": anty_inflammatory_prebiotics,
    "Сорбенты": sorbents,
    }
needed_ferments

{'Ферменты желудочного сока': ['Ацидинпепсин или Энтеросан'],
 'Ферменты поджелудочной железы': ['Панкреатин или Консумед или Эвензим'],
 'Гепатопротекторы и желчегонные': ['Урсодезоксихолевая кислота или Гепатосан'],
 'Антивоспалительный пребиотик': ['Закофальк'],
 'Сорбенты': ['Зостерин Ультра 60']}

In [192]:
preparats = ['Ферменты желудочного сока', 'Ферменты поджелудочной железы',
       'Гепатопротекторы и желчегонные', 'Антивоспалительный пребиотик',
       'Сорбенты']

preparats_dic = {}
for p in preparats:
    if len(needed_ferments[p]) > 0:
        key_name = f'{p}: {needed_ferments[p]} для корректировки этих факторов:'
        preparats_dic[key_name] = kop_bad_df.groupby(p)['Название анализа копрограммы'].apply(list).values[0],

preparats_dic

{"Ферменты желудочного сока: ['Ацидинпепсин или Энтеросан'] для корректировки этих факторов:": (['Щелочной рН кала',
   'Растительная клетчатка перевариваемая'],),
 "Ферменты поджелудочной железы: ['Панкреатин или Консумед или Эвензим'] для корректировки этих факторов:": (['Щелочной рН кала',
   'Жир нейтральный'],),
 "Гепатопротекторы и желчегонные: ['Урсодезоксихолевая кислота или Гепатосан'] для корректировки этих факторов:": (['Растительная клетчатка перевариваемая',
   'Жир нейтральный',
   'Жирные кислоты'],),
 "Антивоспалительный пребиотик: ['Закофальк'] для корректировки этих факторов:": (['Избыток слизи',
   'Растительная клетчатка перевариваемая',
   'Жир нейтральный',
   'Жирные кислоты',
   'Йодофильная флора'],),
 "Сорбенты: ['Зостерин Ультра 60'] для корректировки этих факторов:": (['Щелочной рН кала',
   'Избыток слизи',
   'Растительная клетчатка перевариваемая',
   'Жир нейтральный',
   'Жирные кислоты',
   'Йодофильная флора'],)}

In [206]:
def suggest_ferments_prompt():
    return dedent("""\
    <SYSTEM_MESSAGE>
    Ты Аика - ИИ советник по вопросам здоровья. Ты ведешь диалог с человеком.
    Тебе представлены выбранные пищеварительные ферменты <DRUGS> подобранные профессиональным врачом для коррекции непереваренной пищи, по результатам анализа на копрограмму <LABS>.
    Подробно объясни зачем нужны пищеварительные ферменты, и предложи человеку выбрать ферменты самостоятельно. 
    После чего спроси какие ферменты он выбрал, и есть ли у него вопросы.

    <DRUGS>
    {$DRUGS}
    </DRUGS>
                  
    <LABS>
    {$LABS}
    </LABS>

    </SYSTEM_MESSAGE>
    Ответ:                  
    """
    )

suggest_ferments_prompted = suggest_ferments_prompt()\
    .replace("$LABS", "Результаты копрограммы: " + str(kop1[cols].to_dict(orient='records')))\
    .replace("$DRUGS", str(preparats_dic))
print(suggest_ferments_prompted)

<SYSTEM_MESSAGE>
Ты Аика - ИИ советник по вопросам здоровья. Ты ведешь диалог с человеком.
Тебе представлены выбранные пищеварительные ферменты <DRUGS> подобранные профессиональным врачом для коррекции непереваренной пищи, по результатам анализа на копрограмму <LABS>.
Подробно объясни зачем нужны пищеварительные ферменты, и предложи человеку выбрать ферменты самостоятельно. 
После чего спроси какие ферменты он выбрал, и есть ли у него вопросы.

<DRUGS>
{{"Ферменты желудочного сока: ['Ацидинпепсин или Энтеросан'] для корректировки этих факторов:": (['Щелочной рН кала', 'Растительная клетчатка перевариваемая'],), "Ферменты поджелудочной железы: ['Панкреатин или Консумед или Эвензим'] для корректировки этих факторов:": (['Щелочной рН кала', 'Жир нейтральный'],), "Гепатопротекторы и желчегонные: ['Урсодезоксихолевая кислота или Гепатосан'] для корректировки этих факторов:": (['Растительная клетчатка перевариваемая', 'Жир нейтральный', 'Жирные кислоты'],), "Антивоспалительный пребиотик: ['З

In [225]:
history = [{'role':'user', 'parts': [suggest_ferments_prompted]}]
response = await send_message_to_gemini_async(history, generation_params={"temperature": 0.1})
history.append({'role':'model', 'parts': [response.get('text_response')]})
print(response.get('text_response'))

**Зачем нужны пищеварительные ферменты?**

Пищеварительные ферменты - это белки, которые помогают расщеплять пищу на более мелкие части, которые могут быть усвоены организмом. Они вырабатываются в желудке, поджелудочной железе и тонком кишечнике.

Когда пищеварительная система не вырабатывает достаточного количества ферментов, это может привести к непереваренной пище в стуле. Это может вызвать такие симптомы, как вздутие живота, газы, диарея и боль в животе.

**Какие пищеварительные ферменты вам нужны?**

На основе результатов вашего анализа копрограммы вам могут быть рекомендованы следующие пищеварительные ферменты:

* **Ферменты желудочного сока:** Ацидинпепсин или Энтеросан
* **Ферменты поджелудочной железы:** Панкреатин, Консумед или Эвензим
* **Гепатопротекторы и желчегонные:** Урсодезоксихолевая кислота или Гепатосан
* **Антивоспалительный пребиотик:** Закофальк
* **Сорбенты:** Зостерин Ультра 60

**Пожалуйста, выберите ферменты, которые вы хотели бы принимать:**

**Есть ли у вас

In [226]:
history.append({'role':'user', 'parts': ["как мне выбрать между Ацидинпепсином или Энтеросаном?"]})
response = await send_message_to_gemini_async(history, generation_params={"temperature": 0.1})
history.append({'role':'model', 'parts': [response.get('text_response')]})
print(response.get('text_response'))

**Ацидин-пепсин** и **Энтеросан** - это оба ферментных препарата, которые содержат пепсин и соляную кислоту. Они используются для улучшения пищеварения и устранения симптомов, связанных с недостаточностью желудочной секреции, таких как:

* Вздутие живота
* Газы
* Изжога
* Отрыжка
* Тошнота
* Рвота
* Диарея
* Боль в животе

**Ацидин-пепсин** обычно используется для лечения гипоацидного гастрита, состояния, при котором желудок не вырабатывает достаточного количества желудочного сока. **Энтеросан** также используется для лечения гипоацидного гастрита, а также других состояний, таких как:

* Хронический панкреатит
* Муковисцидоз
* Синдром мальабсорбции
* Послеоперационные состояния

**Выбор между Ацидин-пепсином и Энтеросаном зависит от ваших индивидуальных потребностей и состояния здоровья.** Если у вас гипоацидный гастрит, то оба препарата могут быть эффективны. Однако, если у вас также есть другие состояния, такие как хронический панкреатит или муковисцидоз, то Энтеросан может быть лучш

In [223]:
history = [{'role':'user', 'parts': [suggest_ferments_prompted]}]
response = await send_message_to_gemini_async(history, generation_params={"temperature": 0.1})
history.append({'role':'model', 'parts': [response.get('text_response')]})
print(response.get('text_response'))

**Зачем нужны пищеварительные ферменты?**

Пищеварительные ферменты - это белки, которые помогают расщеплять пищу на более мелкие части, чтобы организм мог их усвоить. Они вырабатываются в желудке, поджелудочной железе и тонком кишечнике.

Когда пищеварительные ферменты не вырабатываются в достаточном количестве или не работают должным образом, это может привести к непереваренной пище в стуле. Это может вызвать такие симптомы, как вздутие живота, газы, диарея и боль в животе.

**Какие пищеварительные ферменты вам нужны?**

На основе результатов вашего анализа копрограммы вам могут быть рекомендованы следующие пищеварительные ферменты:

* **Ферменты желудочного сока:** Ацидинпепсин или Энтеросан
* **Ферменты поджелудочной железы:** Панкреатин, Консумед или Эвензим
* **Гепатопротекторы и желчегонные:** Урсодезоксихолевая кислота или Гепатосан
* **Антивоспалительный пребиотик:** Закофальк
* **Сорбенты:** Зостерин Ультра 60

**Пожалуйста, выберите ферменты, которые вы хотели бы принимать:*

In [224]:
history.append({'role':'user', 'parts': ["как мне выбрать между Ацидинпепсином или Энтеросаном?"]})
response = await send_message_to_openai_async(history, generation_params={"temperature": 0.1})
print(response.get('text_response'))

Оба препарата - Ацидинпепсин и Энтеросан - являются пищеварительными ферментами желудочного сока, которые помогают улучшить процесс пищеварения. Однако они могут иметь различия в составе, дозировке и способе применения.

**Ацидинпепсин** - это комплексный препарат, содержащий желудочный фермент пепсин и соляную кислоту, которые помогают расщеплять белки в пище. Он может быть рекомендован при недостаточной выработке желудочного сока.

**Энтеросан** - это препарат, содержащий ферменты, которые помогают улучшить пищеварение в кишечнике. Он может быть рекомендован при нарушениях пищеварения в тонком кишечнике.

Для выбора между Ацидинпепсином и Энтеросаном рекомендуется обратиться к врачу или фармацевту. Они смогут подробнее рассказать о преимуществах каждого препарата, учитывая ваше состояние здоровья и индивидуальные особенности.

Если у вас есть какие-либо дополнительные вопросы или требуется дополнительная консультация, не стесняйтесь обращаться к специалисту.


In [231]:
history = [{'role':'user', 'parts': [suggest_ferments_prompted]}]
response = await send_message_to_gemini_async(history, generation_params={"temperature": 0.1})
history.append({'role':'model', 'parts': [response.get('text_response')]})
print(response.get('text_response'))

**Зачем нужны пищеварительные ферменты?**

Пищеварительные ферменты - это белки, которые помогают расщеплять пищу на более мелкие части, чтобы организм мог их усвоить. Они вырабатываются в желудке, поджелудочной железе и тонком кишечнике.

Когда пищеварительные ферменты не вырабатываются в достаточном количестве или не работают должным образом, это может привести к непереваренной пище в стуле. Это может вызвать такие симптомы, как вздутие живота, газообразование, диарея и боль в животе.

**Какие пищеварительные ферменты вам нужны?**

На основе результатов вашего анализа копрограммы вам могут потребоваться следующие пищеварительные ферменты:

* **Ферменты желудочного сока:** Ацидинпепсин или Энтеросан
* **Ферменты поджелудочной железы:** Панкреатин, Консумед или Эвензим
* **Гепатопротекторы и желчегонные:** Урсодезоксихолевая кислота или Гепатосан
* **Антивоспалительный пребиотик:** Закофальк
* **Сорбенты:** Зостерин Ультра 60

**Пожалуйста, выберите ферменты, которые вы хотите принимат

In [232]:
history.append({'role':'user', 'parts': ["как мне выбрать между Ацидинпепсином или Энтеросаном?"]})
response = await send_message_open_router_async(history, )
print(response.get('text_response'))

Чтобы выбрать между Ацидинпепсином и Энтеросаном, нужно учитывать следующие факторы:

Ацидинпепсин:
- Содержит фермент пепсин, который вырабатывается в желудке.
- Помогает переваривать белки.
- Лучше принимать во время или сразу после еды, т.к. для его активации требуется кислая среда желудка.

Энтеросан:
- Содержит комплекс пищеварительных ферментов - пепсин, панкреатин, желчные кислоты.
- Помогает переваривать белки, жиры и углеводы.
- Его можно принимать независимо от приема пищи, т.к. содержит компоненты для создания кислой среды.

Если у вас наблюдается преимущественно плохое переваривание белков, лучше выбрать Ацидинпепсин. Если проблемы с перевариванием разных типов пищи, тогда предпочтительнее Энтеросан.

Также имейте в виду потенциальные побочные эффекты, противопоказания и взаимодействия с другими лекарствами, если вы их принимаете. По этим вопросам лучше проконсультироваться с врачом.

Есть ли у вас другие вопросы по выбору фермента или по остальным препаратам?


In [230]:
%debug

> /Users/nikolaytv/workspace/AIka/ZBL_algoritm/llm_api_calls.py(220)send_message_open_router_async()
    218 
    219     if response.status_code == 200 or 'error' not in response.json():
--> 220         return {"text_response": response_text, 
    221                 "input_tokens": input_tokens,
    222                 "output_tokens": output_tokens}

<Response [200]>
{'error': {'message': '{"type":"error","error":{"type":"invalid_request_error","message":"messages: at least one message is required"}}', 'code': 400}}


In [17]:
import pandas as pd
u = pd.DataFrame(usage_history)
print(u['input_tokens'].sum(), u['output_tokens'].sum(), )
u

247 540


,provider,query,answer,input_tokens,output_tokens
0,gemini,Я ознакомился с материалами. Готов продолжить.,Хорошо. Давай начнем с анализа на дисбактериоз...,13,65
1,gemini,Нет значительных отклонений.,Хорошо. А есть ли в анализе превышение количес...,6,104
2,gemini,Уровень энтеробактерий и стафилококков повышен...,Понял. А обнаружены ли в анализе какие-либо па...,23,34
3,gemini,"К счастью, нет, патогенных микроорганизмов не ...",Хорошо. Есть ли у тебя результат на определени...,17,21
4,gemini,"Да, есть.",Отлично. Можешь прислать мне этот результат? Я...,4,23
5,gemini,Уже отправил.,Спасибо. Давай посмотрим. К каким бактериофага...,4,72
6,gemini,S: Enterobacteria phage ΦX174\nR: Pseudomonada...,Хорошо. Теперь давай перейдем к результатам ко...,20,55
7,gemini,Ознакомился с гайдом. Приступайте.,Отлично. Что из перечисленного ниже нашли в тв...,12,161
8,gemini,- Щелочной рН кала - присутствует в умеренном ...,<NEXT_STEP>,148,5


In [25]:

how_to_take_ferments_df()


,Препарат,Применение и дози
0,Ацидинпепсин,"По 500 мг 3-4 раза в сутки, во время или после..."
1,Энтеросан,За 15-20 мин до еды с небольшим количеством во...
2,Панкреатин,"Дозы препарата подбирают индивидуально, в зави..."
3,Консумед,"Дозы препарата подбирают индивидуально, в зави..."
4,Эвензим,"Дозы препарата подбирают индивидуально, в зави..."
5,Гепатосан,"За 15-20 минут до еды, запивая небольшим колич..."
6,Урсодезоксихолевая кислота,По 500 мг перед сном 1 раз в сутки в течение 2...
7,Закофальк,"Взрослым — по 3-4 табл. в день, до еды, не раз..."
8,Зостерин Ультра 60,1г/день 1г - вечером через 2ч после ужина\nКур...


In [27]:
koprogramma_df()

,Результат копрограммы,Ферменты желудочного сока,Ферменты поджелудочной железы,Гепатопротекторы и холекинетики,Анти- и пробиотики,Сорбенты
0,Щелочной рН кала,Ацидинпепсин или Энтеросан,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
1,Избыток слизи,None,None,None,Закофальк,Зостерин Ультра 60
2,Остатки непереваренной пищи,Ацидинпепсин или Энтеросан,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
3,Мышечные волокна неизмененные (с исчерченностью),Ацидинпепсин или Энтеросан,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
4,Растительная клетчатка перевариваемая,Ацидинпепсин или Энтеросан,None,Урсодезоксихолевая кислота или Гепатосан,Закофальк,Зостерин Ультра 60
5,Жир нейтральный,None,Панкреатин или Консумед или Эвензим,Урсодезоксихолевая кислота или Гепатосан,Закофальк,Зостерин Ультра 60
6,Жирные кислоты,None,None,Урсодезоксихолевая кислота или Гепатосан,Закофальк,Зостерин Ультра 60
7,Мыла,None,Панкреатин или Консумед или Эвензим,None,None,Зостерин Ультра 60
8,Краxмал,Ацидинпепсин или Энтеросан,None,None,None,Зостерин Ультра 60
9,Краxмал внутриклеточный,None,Панкреатин или Консумед или Эвензим,None,Закофальк,Зостерин Ультра 60


In [23]:
print(koprogramma_df()['Результат копрограммы'].values)

['Щелочной рН кала' 'Избыток слизи' 'Остатки непереваренной пищи'
 'Мышечные волокна неизмененные (с исчерченностью)'
 'Растительная клетчатка перевариваемая' 'Жир нейтральный'
 'Жирные кислоты' 'Мыла' 'Краxмал' 'Краxмал  внутриклеточный'
 'Крахмал внеклеточный' 'Кристаллы триппельфосфаты'
 'Кристаллы Шарко–Лейдена' 'Йодофильная флора' 'Клостридии' 'Эпителий']


# USAGES

In [10]:
# SONNET
import pandas as pd
us = pd.DataFrame(usage_history)
print("us['input_tokens'].sum()", us['input_tokens'].sum())
print("us['output_tokens'].sum()", us['output_tokens'].sum())
print("price", us['input_tokens'].sum()/1000 * 0.003 + us['output_tokens'].sum()/1000 * 0.015)
us

us['input_tokens'].sum() 15205
us['output_tokens'].sum() 1016
price 0.060855000000000006


,provider,query,answer,input_tokens,output_tokens
0,router_sonnet,"Здравствуйте, Аика. К сожалению, у меня нет пр...","Хорошо, в таком случае давайте продолжим наш д...",1206,52
1,router_sonnet,"Максим, 32 года.","Отлично, Максим. Расскажите, каких целей по ул...",1270,46
2,router_sonnet,Повысить общий уровень здоровья и продолжить в...,"Понятно, отличная цель! Давайте начнем с оценк...",1347,103
3,router_sonnet,Нормотоники,"Хорошо, спасибо за информацию. Следующий важны...",1459,118
4,router_sonnet,"Да, делал УЗИ органов брюшной полости и желчно...","Отлично, это хорошие результаты. Теперь скажит...",1640,98
5,router_sonnet,"Вы правы, доктор. Я выбрала тысячелистник для ...","Замечательно, Максим! Ваш выбор тысячелистника...",1774,172
6,router_sonnet,"Камней нет, я буду выполнять гимнастику вечеро...",Отличный выбор времени для желчегонной гимнаст...,1992,149
7,router_sonnet,"Да, сдавал. Результат показал 29 нг/мл. Приним...","Понятно, спасибо за информацию о витамине D. 2...",2185,135
8,router_sonnet,"Утром, перед работой.","Отлично, принятие лимфатического душа утром пе...",2332,143


In [14]:
# OPENAI
import pandas as pd
us = pd.DataFrame(usage_history)
print("us['input_tokens'].sum()", us['input_tokens'].sum())
print("us['output_tokens'].sum()", us['output_tokens'].sum())
print("price", us['input_tokens'].sum()/1000 * 0.01 + us['output_tokens'].sum()/1000 * 0.03)
us

us['input_tokens'].sum() 20196
us['output_tokens'].sum() 1523
price 0.24765000000000004


,provider,query,answer,input_tokens,output_tokens
0,gpt-4-turbo-2024-04-09,L847-LP9B-8KY63.,"Спасибо за предоставление промокода! Теперь, д...",1241,84
1,gpt-4-turbo-2024-04-09,"Меня зовут Джессика, мне 32 года.","Приятно познакомиться, Джессика! Теперь давайт...",1351,89
2,gpt-4-turbo-2024-04-09,{Я хотела бы улучшить свою физическую форму и ...,"Отличные цели, Джессика! Улучшение физической ...",1528,191
3,gpt-4-turbo-2024-04-09,"У меня нормальное артериальное давление, 120/80.","Спасибо за информацию, Джессика. Ваше давление...",1749,184
4,gpt-4-turbo-2024-04-09,Нормокинетический.,"Отлично, что у вас нормокинетический тип сокра...",1950,137
5,gpt-4-turbo-2024-04-09,"Да, тысячелистник.","Прекрасный выбор, Джессика! Тысячелистник отли...",2106,144
6,gpt-4-turbo-2024-04-09,"Да, УЗИ брюшной полости показало отсутствие ка...","Это отличные новости, Джессика! Отсутствие кам...",2296,142
7,gpt-4-turbo-2024-04-09,Вечером.,"Отличный выбор времени, Джессика. Вечером упра...",2451,156
8,gpt-4-turbo-2024-04-09,"Да, сдавала. Планирую принимать 2000 МЕ в день.","Хорошо, что вы контролируете уровень витамина ...",2641,229
9,gpt-4-turbo-2024-04-09,Вечером.,"Прекрасно, вечернее время идеально подходит дл...",2883,167


In [54]:
# GEMINI
us.to_csv('gemini_token_usage_16msgs.csv')
us

,input_tokens,output_tokens,conversation_tokens
0,979,47,1026
1,18,30,1074
2,4,51,1129
3,11,39,1179
4,14,87,1280
5,3,60,1343
6,5,86,1434
7,6,54,1494
8,4,141,1639
9,17,108,1764
